In [12]:
from abc_utils import *
import pandas as pd
from hmmlearn import hmm
import numpy as np

In [13]:
train_set, train_lengths, val_set, _ = load_datasets()

In [14]:
def load_song(index):
    start_pos = train_lengths.iloc[:index].sum().item()
    return train_set.iloc[start_pos : start_pos + train_lengths.iloc[index].item()]

In [15]:
def load_songs(indices):
    end_positions = np.cumsum(train_lengths)
    positions = np.insert(end_positions, 0, np.array([0]))
    songs = []
    for i in indices:
        song = train_set.iloc[positions[i] : positions[i+1]]
        songs.append(song)
    # return songs
    return pd.concat(songs)

song_indices = list(range(1))
songs = load_songs(song_indices)

In [16]:
states, observations = dataframe_to_states(songs, 1, 1)

Processing states: 100%|██████████| 104/104 [00:00<00:00, 4222.07it/s]


In [17]:
transition_matrix, emission_probs, unique_states, unique_obs, states_to_index, observation_to_index = states_to_transition(states, observations)

In [18]:
transition_matrix.shape

(35, 35)

In [19]:
emission_probs.shape

(11, 35)

In [20]:
unique_obs

array([[70.],
       [72.],
       [74.],
       [75.],
       [77.],
       [79.],
       [80.],
       [82.],
       [84.],
       [86.],
       [87.]])

In [22]:
observation_to_index

{(np.float64(70.0),): 0,
 (np.float64(72.0),): 1,
 (np.float64(74.0),): 2,
 (np.float64(75.0),): 3,
 (np.float64(77.0),): 4,
 (np.float64(79.0),): 5,
 (np.float64(80.0),): 6,
 (np.float64(82.0),): 7,
 (np.float64(84.0),): 8,
 (np.float64(86.0),): 9,
 (np.float64(87.0),): 10}

In [23]:
observation_indices = np.array([observation_to_index[(o,)] for o in observations]).reshape(-1, 1)

In [25]:
observation_indices

array([[ 7],
       [ 6],
       [ 5],
       [ 4],
       [ 5],
       [ 3],
       [ 3],
       [ 2],
       [ 3],
       [ 4],
       [ 0],
       [ 2],
       [ 3],
       [ 4],
       [ 3],
       [ 4],
       [ 5],
       [ 4],
       [ 5],
       [ 7],
       [ 6],
       [ 5],
       [ 4],
       [ 5],
       [ 3],
       [ 3],
       [ 2],
       [ 3],
       [ 4],
       [ 0],
       [ 2],
       [ 4],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [10],
       [ 9],
       [10],
       [ 7],
       [ 7],
       [ 6],
       [ 5],
       [ 4],
       [ 3],
       [ 2],
       [ 1],
       [ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 7],
       [10],
       [ 9],
       [10],
       [ 7],
       [ 7],
       [ 6],
       [ 5],
       [ 4],
       [ 3],
       [ 2],
       [ 4],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 4],
       [ 5],
       [ 6],

In [27]:
model = hmm.CategoricalHMM(
    n_components=transition_matrix.shape[0],
    init_params = ''
)

model.fit(observation_indices.reshape(-1, 1), lengths=train_lengths.iloc[song_indices])

model.transmat = transition_matrix.T
model.emissionprob = emission_probs.T
start_probs = np.ones(transition_matrix.shape[0]) / transition_matrix.shape[0]
model.startprob = start_probs

Fitting a model with 1574 free scalar parameters with only 104 data points will result in a degenerate solution.


KeyError: -1

In [34]:
model.decode(observations, lengths=train_lengths.iloc[song_indices])

NotFittedError: This CategoricalHMM instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.